## Упражнение 04: A/B-тестирование

In [1]:
import pandas as pd
import sqlite3

## создаем подключение к базе данных с помощью библиотеки sqlite3

In [2]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

## используя только один запрос для каждой из групп, создайте два фрейма данных: test_results и control_results со столбцами time и avg_diff и только двумя строками

- время должно иметь значения: после и до
- avg_diff содержит среднюю дельту среди всех пользователей за период времени до того, как каждый из них совершил свой первый визит на страницу, и после этого
- учитывать только тех пользователей, у которых есть наблюдения до и после

In [3]:
query = """
        with all_delts as
        (select uid, first_view_ts, first_commit_ts,
        datetime(deadlines.deadlines, 'unixepoch')      as deadline,
        (deadlines - unixepoch(first_commit_ts)) / 3600 as delta_commit_deadline,
        case
            when first_view_ts <= first_commit_ts then 'after'
            else 'before' end as time
        from test
        join deadlines on deadlines.labs = test.labname
        where labname != 'project1')
        select time, avg(delta_commit_deadline) as avg_diff from all_delts
        group by time
        """

test_results = pd.read_sql(query, conn)
test_results

,time,avg_diff
0,after,103.40625
1,before,60.56250


In [4]:
query = """
        with all_delts as
        (select uid, first_view_ts, first_commit_ts,
        datetime(deadlines.deadlines, 'unixepoch')      as deadline,
        (deadlines - unixepoch(first_commit_ts)) / 3600 as delta_commit_deadline,
        case
            when first_view_ts <= first_commit_ts then 'after'
            else 'before' end as time
        from control
        join deadlines on deadlines.labs = control.labname
        where labname != 'project1')
        select time, avg(delta_commit_deadline) as avg_diff from all_delts
        group by time
        """

control_results = pd.read_sql(query, conn)
control_results

,time,avg_diff
0,after,112.710526
1,before,99.464286


## закрыть соединение

In [5]:
conn.close()